#  Experimenting different Techniques

In [ ]:
Sources for datasets

- liftered from intresting git hub

In [ ]:
# Import libaries
import pandas as pd
import numpy as np
import json
import copy
import os
import math
from math import sqrt
import matplotlib.pyplot as plt
import statistics
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import LassoLars
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import randint as sp_randint
from IPython import get_ipython

In [ ]:
import warnings
warnings.filterwarnings('ignore')

get_ipython().run_line_magic("matplotlib", "inline")

In [ ]:
def user_input():
    forecast_period = input("What is the Future Forecast period? ")
    forecast_period = int(forecast_period)

    input_data = input("Enter dataset: ")


    file_type = os.path.splitext(input_data)[1]
    if file_type == '.csv':
        dataset = pd.read_csv(input_data)

    elif file_type == '.json':
        dataset = retrieve_data(input_data)

    return forecast_period, dataset

def retrieve_data(path):
    datasets = dict()
    with open(path) as json_file:
        raw_data = json.load(json_file)

        for element in raw_data:
            df_name = element['sku']
            data = pd.DataFrame()
            data['time'] = element['time']
            data['sales'] = element['sales']
            data['sales'] = [np.nan if pd.isnull(i) else int(i) for i in data['sales']]

            data = data.T
            data = data.rename(columns = data.iloc[0]).drop(data.index[0])
            datasets[df_name] = copy.deepcopy(data)

    return datasets

In [ ]:

def dickeyfullertest(series):
    dftest = adfuller(series, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index= ['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])

    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value

    if dfoutput['p-value'] > 0.05:
        return 0 #not stationary

    else:
        return 1 #stationary

def mean_standard_deviation(dataset):
    mu = np.mean(dataset.values)
    sd = np.std(dataset.values)

    ub = mu + (3 * sd)
    lb = mu - (3* sd)

    return lb,ub

In [ ]:

def acf_plot1(dataset,freq):
#    plot_acf(dataset)
    res = acf(dataset)
    ub=1.96/np.sqrt(len(dataset))
    for i in range(1, len(res)-1):
        if(res[i] > ub and res[i + 1] < ub):
            p = i
            if (p > freq):
                p = freq
            break
    else:
        p = freq
    return p

def acf_plot(dataset,freq):
    res = acf(dataset)
    plot_acf(dataset)
    acfval=[0]*len(res)
    ub=1.96/np.sqrt(len(dataset))
    p1=1
    for i in range(len(res)):
        acfval[i]=abs(res[i])
    acfval.sort(reverse=True)
    acfval=np.array(acfval[1:])
    pshort=np.array(acfval[0:3])
    pshortind=[0]*len(pshort)
#    print(pshort)
    for i in range(len(pshort)):
        pshortind[i]=np.where(abs(res)==pshort[i])[0][0]
    ind=np.where(acfval>ub)[0]
    finalacf=acfval[ind]
    plist=[0]*len(finalacf)
    for i in range(len(finalacf)):
        plist[i]=np.where(abs(res)==finalacf[i])[0][0]
#    return pshortind,plist
#    print(plist)
    while len(finalacf)>0:
        p1=np.where(abs(res)==max(finalacf))[0][0]
        if p1 > freq:
            finalacf=finalacf[1:]
        else:
            break
    return p1,pshortind,plist

def pacf_plot1(dataset,freq):
#    plot_pacf(dataset)
    res = pacf(dataset)
    ub=1.96/np.sqrt(len(dataset))
    for i in range(1, len(res)-1):
        if(res[i] > ub and res[i + 1] < ub):
            q = i
            if (q > freq/2):
                q = freq/2
            break
    else:
        q = freq/2
    return int(q)

def pacf_plot(dataset,freq):
    res = pacf(dataset)
    plot_pacf(dataset)
    pacfval=[0]*len(res)
    ub=1.96/np.sqrt(len(dataset))
    q1=0
    for i in range(len(res)):
        pacfval[i]=abs(res[i])
    pacfval.sort(reverse=True)
    pacfval=np.array(pacfval[1:])
    ind=np.where(pacfval>ub)[0]
    finalpacf=pacfval[ind]
    while len(finalpacf)>0:
        q1=np.where(abs(res)==max(finalpacf))[0]
        q1=q1[0]
        if q1 > int(freq/2):
            finalpacf=finalpacf[1:]
        else:
            break
    return q1


In [ ]:
#TODO: Send alpha, beta
def median_absolute_deviation(dataset, median):
    median_list = list()
    dataset.reset_index(drop = True, inplace = True)
    for i in range(0, len(dataset)):
        value = dataset.T[i] - median
        median_list.append(value)
    ms = np.abs(median_list)
    mad = np.median(ms)
    ub = median + (3 * mad)
    lb = median - (3 * mad)


    return ub,lb

def dateformat(dataset):
    dataset['time'] = dataset.index.tolist()
    dataset['time']=pd.to_datetime(dataset['time'], format='%m/%d/%y',infer_datetime_format=True)
    dataset.set_index('time', inplace = True)
    return dataset

def outlier_treatment(dataset):
    median = np.median(dataset)
    if median == 0:
        ub,lb = mean_standard_deviation(dataset)
    else:
        ub,lb = median_absolute_deviation(dataset, median)
    new_dataset = np.clip(dataset, lb, ub)
    return new_dataset

#pt--->outlier bucket size
#sindex ---> number of zeros to categorize as sparse data
#freq ---> seasonality
def get_bucket_size(interval):
    interval_type = find_interval_type(interval) #aggregation (weekly/monthly)
    if interval_type == 'W':
        pt=12
        sindex=24
        freq=52
    elif interval_type=='M'or interval_type=='Random':
        pt=6
        sindex=10
        freq=12
    elif interval_type=='Y':
        pt=2
        sindex=0
        freq=0
    return pt,sindex,freq

def outlier_treatment_tech(dataset,interval,pt): 
    start=0
    end=pt
    sku_data=[0]*len(dataset)
    while end < len(dataset):
        sku_data[start:end]=outlier_treatment(dataset[start:end])
        start=end
        end+=pt
    if start < len(dataset):
        sku_data[start:len(dataset)]=outlier_treatment(dataset[start:end])
    sku_data=pd.DataFrame(sku_data)
    return sku_data

def Sesonal_detection(sku_data):
    median = np.median(sku_data)

    if median == 0:
        ub,lb = mean_standard_deviation(sku_data)
    else:
        ub,lb = median_absolute_deviation(sku_data, median)
    outliers1= sku_data > ub
    outliers2 = sku_data < lb
    a=np.where(outliers1==True)[0]
    b=np.where(outliers2==True)[0]
    flag1=flag2=1
    if len(a)==0:
        flag1=0
        remove1=[]
    if len(b)==0:
        flag2=0
        remove2=[]

    if flag1==1:
        k=np.zeros([len(a)-1, len(a)])
        for i in range(0,(len(a)-1)):
            for j in range(1,len(a)):
                if a[j]==(a[i]+12) or a[j]==(a[i]+24):
                    k[i][j]=1
                else:
                    k[i][j]=0
        m=np.where(k!=0)
        z=np.unique(m).tolist()
        remove1=a[z]
    if flag2==1:
        q=np.zeros([len(b)-1, len(b)])
        for i in range(0,(len(b)-1)):
            for j in range(1,len(b)):
                if b[j]==(b[i]+12) or b[j]==(b[i]+24):
                    q[i][j]=1
                else:
                    q[i][j]=0
        n=np.where(q!=0)
        z1=np.unique(n).tolist()
        remove2=b[z1]
    return remove1,remove2,flag1,flag2

def impute_missing_dates(dataset):
    interval,start_date,end_date=find_interval(dataset.index)
    drange = pd.date_range(start_date, end_date, freq = interval)
    comp_data = pd.DataFrame(drange, columns = ['time'])
    sales=dataset['sales'].to_dict()
    comp_data['sales'] = comp_data['time'].map(sales)
    comp_data.drop('time', axis = 1, inplace = True)
    return comp_data, interval

def find_interval(date):
    date=pd.to_datetime(date, format='%m/%d/%Y',infer_datetime_format=True)
    diff=[]
    for i in range(len(date)-1):
        interval=date[i+1]-date[i]
        diff.append(interval)
    mode=statistics.mode(diff)

    return mode,date[0],date[-1]


def find_interval_type(interval):
    interval=interval.days
    if interval==7:
        itype='W'
    elif interval==30 or interval==31:
        itype='M'
    elif interval==365:
        itype='Y'
    else:
        itype='Random'

    return itype

def data_imputation_zero(dataset):
    dataset.fillna(0, inplace = True)
    return dataset

#TODO: Send as transpose of dataset
def data_imputation(dataset,freq):
    #Taking the mean of nearest neighbours to fill NA

    data_forward = dataset.fillna(method = 'ffill')
    data_back = dataset.fillna(method = 'bfill')
    data_back.fillna(0, inplace = True)
    data_forward.fillna(0, inplace = True)

    new_data = (data_forward.values + data_back.values) / 2
    dataset=pd.DataFrame(dataset.values)

    imput=dataset.isnull()
    imput=imput[0]

    dataset=dataset[0]
    for i in range(len(dataset)):
        div_factor = 3
        if imput[i]==True:
            #Negative index, set previous as 0
            if i - freq < 0:
                prev_value = 0
                div_factor -= 1
            #Fetch previous value
            else:
                prev_value = dataset[i - freq]

            #Outside boundary or next value is NaN, set previous as 0
            if i + freq >= len(dataset) or imput[i + freq]==True:
                next_value = 0
                div_factor -= 1
            else:
                next_value = dataset[i + freq]
            dataset[i] = (new_data[i] + prev_value + next_value)/div_factor

    df = pd.DataFrame(dataset)
    return df

#reading from first non-zero
def read_from_first_sales(sku_data):
    test=pd.isnull(sku_data)
    index=np.where(test==False)[0]
    index=index[0]
    sku_data = sku_data[index:]
    sku_data=sku_data.reset_index(drop = True)
    return sku_data

def new_forecast(data,forecast,forecast_period):

    sample_data=data[-forecast_period:]
    sample_data=pd.DataFrame(sample_data)
    forecast=pd.DataFrame(forecast)
    median_data=np.median(sample_data)
    median_fore=np.median(forecast)
    if median_data == 0:
        ub_d,lb_d = mean_standard_deviation(sample_data)
    else:
        ub_d,lb_d = median_absolute_deviation(sample_data, median_data)
    if median_fore == 0:
        ub_f,lb_f = mean_standard_deviation(forecast)
    else:
        ub_f,lb_f = median_absolute_deviation(forecast, median_fore)
    if ub_f >ub_d:
        ub=ub_d
    else:
        ub=ub_f
    if lb_f >lb_d:
        lb=lb_d
    else:
        lb=lb_f
    forecast=np.clip(forecast,lb,ub)
    return forecast



'''
Function used to fit the model
'''

def fit_model(train_data, model):

    X, y = train_data[:, 0:-1], train_data[:, -1]
    model.fit(X, y)
    return model


'''
Function used for one-step forecasting
'''

def forecast_model(model, X):

    yhat = model.predict(X)

    return yhat


'''
Function used for plotting

'''
def plotting(key, predictions, expected):
    rmse = calculate_rmse(key, expected, predictions)
    plt.figure()
    plt.title(key)

    y_values = list(expected) + list(predictions)
    y_range = max(y_values) - min(y_values)
    plt.text(6, min(y_values) + 0.2 * y_range, 'RMSE = ' + str(rmse))
    plt.plot(predictions)
    plt.plot(expected)
    plt.legend(['predicted', 'expected'])
    plt.show()


def calculate_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = sum(np.abs((y_true - y_pred) / y_true)* 100)/len(y_true)
    if np.isnan(mape)== True or np.isfinite(mape)==False:
        mape=0
    return mape


def calculate_facc(y_true, y_pred):

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    facc = 1 - (sum(np.abs(y_true - y_pred)) / sum(y_true))
    if np.isnan(facc)== True or np.isfinite(facc)==False:
        facc=0
    return facc * 100



def calculate_rmse(key, expected, predictions):
    expected=np.array(expected)
    rmse = sqrt(mean_squared_error(expected, predictions))
    print("RMSE FOR %s: %d " % (key, rmse))
    return rmse

